# Import 

In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy

In [2]:
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## Initiate BratDirReader

In [3]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [4]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [5]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read Brat annotations | load from pickles

In [6]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [7]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


# Define sampling functions

In [8]:
rounds =10
seed= 14

In [9]:
len(train_docs), len(test_docs)

(303, 202)

### Define CRF Wrapper

In [10]:
from CRFWrapper import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel
from ALSampler import SamplingSimulator, ModelSamplingSimulator

In [11]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Frequency', 'Dosage', 'Drug', 'Strength', 'Route', 'Reason', 'Duration', 'ADE', 'Form'}


In [12]:
crf_model=CRFModel(anno_types=annos)
m_simulator=ModelSamplingSimulator(train_docs, total_round=10, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed)

### Test run

In [13]:
scores=m_simulator.simulate_rounds()

2024-05-22 11:41:34.116 | INFO     | ALSampler:simulate_rounds:53 - simulate round 0.
2024-05-22 11:41:34.117 | DEBUG    | ALSampler:keep_sample:39 - The first round sampling will be random
2024-05-22 11:41:34.119 | INFO     | ALSampler:keep_sample:43 - current sampled: 30, remaining: 273
2024-05-22 11:41:38.138 | DEBUG    | CRFWrapper:fit:266 - Reset and train CRF model...
2024-05-22 11:42:37.656 | DEBUG    | CRFWrapper:fit:274 - Training complete.
2024-05-22 11:42:37.750 | DEBUG    | CRFWrapper:bootstrap_eval:299 - Predicting eval docs...
2024-05-22 11:43:17.190 | DEBUG    | CRFWrapper:bootstrap_eval:303 - Calculate scores from bootstrapping 200 times
2024-05-22 11:47:43.647 | DEBUG    | CRFWrapper:bootstrap_eval:313 - complete
2024-05-22 11:47:43.786 | DEBUG    | ALSampler:simulate_rounds:64 - {'precision': 0.9436780861622498, 'recall': 0.7790147749120776, 'f1': 0.8534724728682227}
2024-05-22 11:47:43.790 | INFO     | ALSampler:simulate_rounds:53 - simulate round 1.
2024-05-22 11:47

In [14]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)


In [15]:
pd.options.display.float_format='{:,.4f}'.format

In [16]:
pd.DataFrame(summary)

,precision,pl,pu,recall,rl,ru,f1,fl,fu
0,0.9437,0.9399,0.9471,0.7790,0.7721,0.7856,0.8535,0.8489,0.8579
1,0.9454,0.9421,0.9485,0.8093,0.8027,0.8151,0.8721,0.8674,0.8762
2,0.9452,0.9414,0.9484,0.8272,0.8217,0.8329,0.8822,0.8782,0.8865
3,0.9461,0.9431,0.9494,0.8371,0.8314,0.8433,0.8883,0.8847,0.8928
4,0.9448,0.9417,0.9481,0.8444,0.8389,0.8496,0.8918,0.8879,0.8958
5,0.9444,0.9408,0.9478,0.8527,0.8477,0.8583,0.8962,0.8926,0.9000
6,0.9471,0.9437,0.9502,0.8529,0.8478,0.8583,0.8976,0.8936,0.9011
7,0.9450,0.9418,0.9483,0.8572,0.8524,0.8629,0.8990,0.8953,0.9027
8,0.9482,0.9448,0.9511,0.8571,0.8522,0.8625,0.9003,0.8965,0.9040
9,0.9459,0.9427,0.9490,0.8596,0.8551,0.8649,0.9007,0.8972,0.9046


## Bootstrap 3 different initials runs

In [17]:
boostrap_runs=3
total_round=10

In [18]:
logger.remove()
logger.add(sys.stderr, level='INFO')

1

In [19]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [20]:
all_scores=[]
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    crf_model=CRFModel(anno_types=annos)
    m_simulator=ModelSamplingSimulator(train_docs, total_round=total_round, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed)
    scores=m_simulator.simulate_rounds()
    all_scores.append(scores) 

2024-05-22 13:42:53.116 | INFO     | __main__:<module>:3 - start run 0.
2024-05-22 13:42:53.125 | INFO     | ALSampler:simulate_rounds:53 - simulate round 0.
2024-05-22 13:42:53.127 | INFO     | ALSampler:keep_sample:43 - current sampled: 30, remaining: 273
2024-05-22 13:48:27.179 | INFO     | ALSampler:simulate_rounds:53 - simulate round 1.
2024-05-22 13:49:41.466 | INFO     | ALSampler:keep_sample:43 - current sampled: 60, remaining: 243
2024-05-22 13:56:55.923 | INFO     | ALSampler:simulate_rounds:53 - simulate round 2.
2024-05-22 13:57:58.822 | INFO     | ALSampler:keep_sample:43 - current sampled: 90, remaining: 213
2024-05-22 14:06:14.908 | INFO     | ALSampler:simulate_rounds:53 - simulate round 3.
2024-05-22 14:07:09.660 | INFO     | ALSampler:keep_sample:43 - current sampled: 120, remaining: 183
2024-05-22 14:16:27.401 | INFO     | ALSampler:simulate_rounds:53 - simulate round 4.
2024-05-22 14:17:14.261 | INFO     | ALSampler:keep_sample:43 - current sampled: 150, remaining: 

In [22]:
joblib.dump(all_scores, r'..\data\vb_sampling\ner_crf_scores.joblib')

['..\\data\\vb_sampling\\ner_crf_scores.joblib']